In [1]:
from dotenv import load_dotenv
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from Levenshtein import distance as levenshtein_distance
from array import array
import os
from PIL import Image
import sys
import time

load_dotenv()

subscription_key = os.environ.get('AZURE_SUBSCRIPTION_KEY')
endpoint = os.environ.get('AZURE_ENDPOINT')

cv_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))


def text_from_image(image_path, mode="Handwritten"):
    with open(image_path, "rb") as image_file:
        read_response = cv_client.read_in_stream(
            image=image_file,
            mode="Printed",
            raw=True
        )
    operation_id = read_response.headers['Operation-Location'].split('/')[-1]
    while True:
        read_result = cv_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)
    result = []
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                result.append(line.text)
        return ' '.join(result)
 
 
# calculare levstein 
#   D[i,j] = min(
#   D[i-1,j] + 1,            // ștergere
#   D[i,j-1] + 1,            // inserare
#   D[i-1,j-1] + cost        // substituire sau nicio modificare
#)      
def character_quality(recognized, org):
    recognized = recognized.strip()
    org = org.strip()

    m, n = len(recognized), len(org)
    matrice = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    for i in range(m + 1):
        matrice[i][0] = i
    for j in range(n + 1):
        matrice[0][j] = j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if recognized[i - 1] == org[j - 1]:
                cost = 0
            else:
                cost = 1
            matrice[i][j] = min(matrice[i - 1][j] + 1, matrice[i][j - 1] + 1, matrice[i - 1][j - 1] + cost)

    lev_distance = matrice[m][n]

    print(f"lev_distance: {lev_distance}")



def word_quality(recognized, org):
    recognized = recognized.strip()
    org = org.strip()

    recognized_words = recognized.split()
    org_words = org.split()

    m, n = len(recognized_words), len(org_words)
    matrice = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    for i in range(m + 1):
        matrice[i][0] = i
    for j in range(n + 1):
        matrice[0][j] = j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if recognized_words[i - 1] == org_words[j - 1]:
                cost = 0
            else:
                cost = 1
            matrice[i][j] = min(matrice[i - 1][j] + 1, matrice[i][j - 1] + 1, matrice[i - 1][j - 1] + cost)

    lev_distance = matrice[m][n]
    print(f"lev_distance: {lev_distance}")



recognized_text = text_from_image("test2.jpeg", "Printed")
original = "Succes în rezolvarea TEMELOR la LABORATOARELE de Inteligență Artificială!"
character_quality(recognized_text, original)
print(recognized_text)
word_quality(recognized_text, original)
    

lev_distance: 14
Lucces in resolvarea TEMELOR la LABORA toarele de Inteligenta Artificialà!
lev_distance: 7
